# **Simulated Experiment:** Calibration in the Presence of Flux Suppression
### **Author:** Brian Welman
### **Date:** June 2023
---
## Purpose of this Notebook

This Jupyter notebook serves as a companion to my Master of Science thesis, *"Calibration with Kalman Filtering and Smoothing: A New Framework for Modernizing Calibration Techniques"*. It provides a simulated environment to investigate and run an experiment focusing on the phenomenon of flux suppression encountered during radio interferometry.

The simulation is combined with a generic, direction-independent, time-only calibration scenario, with the primary objective being to generate the necessary results for the application section of my thesis.

---
## What You Will Find

In this notebook, you'll find a detailed walkthrough of:

1. Simulating a calibration environment
2. Conducting multiple calibration runs
3. Performing a specialized form of imaging
4. Generating plots and FITS files for results visualization

---
## Before You Begin

Please ensure you've checked the README in the GitHub repository for instructions on setting up and running the simulation.

<div class="alert alert-block alert-success"><b>Note:</b> This simulation is intended to serve as an academic example and may not be applicable for all calibration scenarios encountered in radio interferometry. Feel free to adapt and modify the code as needed to suit your specific requirements.</div>

<div class="alert alert-block alert-warning">
<b>Important:</b> As a redundancy feature, all important files and folders, including configuration files, data files/folders, plots, and so forth, that will be generated for a given function call or cell execution will prompt a response from the user whether the original files should be replaced, if they exist. There are several places where this feature can be manually turned off and will be labelled accordingly.
</div>


---
## 1. Simulation Setup

The contents of this section relates to the setup and settings specific to this simulation. Each simulation notebook can facilitate multiple different simulations at the same time, however, it is suggested to keep one simulation per simulation notebook to avoid complications. 

### 1.1 Configure `IPython` Modules

Below loads and sets the various aspects related to the `IPython` module.

In [2]:
# Load magic modules
%reload_ext autoreload

# Auto-reload python code before each execution
%autoreload 2

### 1.2 Configure Notebook

The code below assists with `jupyter-notebook` related conifgurations. 

<div class="alert alert-block alert-warning">
<b>Important:</b> The notebook works best when the screen width is set wider, which can be done by executing the cell below. If the outputs are cleared, you can simply rerun the cell below to restore the width.
</div>


In [3]:
from IPython.display import display, HTML

# Make the cells the width of the screen
display(HTML("<style>.container { width:100% !important; }</style>"))

### 1.3 Create or Load Settings

This sets the core settings for the simulation. That is, the specific locations/paths, labels, parameters, and options for the various aspects of the simulation. The user is welcome to change them as they like. It allows for the persistence of certain parameters and settings per simulation.

<div class="alert alert-block alert-danger">
<b>Caution:</b> If changes are made to the settings, please press the <b>save</b> file button to retain them. If you want to load settings, simply press the <b>load</b> button and the settings should be loaded (given that it is present).
</div>

Below, the main entry point for the script is given by `script_settings` function. It takes the path to the main configuration file for the simulation as a YAML file. If you created a new simulation, please find the relevant configuration YAML file contained within the simulation with the associated name used to generate the directory. If you are using the main repository as your simulation location, you can simply use the file `meerkat-src-100-config.yml` as a start point. This file ensures that the correct simulation is selected and the correct settings, data and parameters associated with the simulation are used. 

<div class="alert alert-block alert-success"><b>Note:</b> Replace the <code>"&lt;INSERT CONFIG SCRIPT HERE&gt;"</code> with a string input of the path to the configuration file.</div>



In [1]:
from source.script import script_settings

script_options = script_settings("<INSERT CONFIG SCRIPT HERE>")

Once the script settings are to your liking, they can be fed into the `setup_simulation` function to create the relevant directories and files according to these settings. From this point onwards, a log file is generated and stored in main directory labelled `simulation.log`. It will detail all the function calls used throughout the script and other useful debugging information.

<div class="alert alert-block alert-success"><b>Note:</b> Feed <code>script_options</code> into <code>setup_simulation</code>. There is an additional flag parameter <code>cleanup</code> that can be set. If set, then existing directories and files will be flagged for deletion, but for caution, it will require a response whether this action should be done or not.</div>

<div class="alert alert-block alert-danger">
    <b>Caution:</b> Regardless whether the <code>cleanup</code> flag is set, running this function will prompt for deletion of parameter information if any existing parameters that is stored in <code>"&lt;config_dir&gt;/params.config"</code> is present, where <code>config_dir</code> is the path to your configuration file directory given in the script settings. This is important to not delete if you wish to keep your simulation global parameters alive per session. If this is the case, select <b>n</b> everytime. If are completely deleting and reseting the simulation or this information in particular, then select <b>y</b>. If this is done, rerun all relevant functions below without replacing existing files. This will update the parameter information with associated settings and function calls.
</div>


In [5]:
from source.script import setup_simulation

params = setup_simulation(script_options)

The output of `setup_simulation`, i.e., `params` is the most important parameter information file within the script. It will contain all the deduced user settings and other important information to be used throughout the simulation. This can range from how the data is simulated, where to find certain data files/folders, allowing for calibration and imaging runs, and performing the necessary actions to create plots.

The variable can be accessed like a Python dictionary through the use of keys. It will be up to the user to find the required information they need, but it is designed for easy viewing if the variable is printed to screen, i.e., `print(params)`. The user is also welcome to store data within the parameter file if need be. If this information is only required for a session or cell, then you can use it as a normal dictionary in this context. If you wish to save this custom parameter information, execute `params.save()` to push all the new changes to file. 

If a function call says there is missing keys, it is often easier to rerun previous function calls (without deleting the generated data and settings) to correct this, e.g., `setup_plotting` to reset the plotting information, `create_empty_measurement_set` to reset measurement set information, or 

<div class="alert alert-block alert-danger">
    <b>Caution:</b> When assigning new variables to <code>params</code> or saving the custom entries to file, there is no check done to see if things are overwritten. It is suggested to use a separate entry as <code>params["custom"] = {}</code> and populate this dictionary with the user's custom parameter information. Ensure to call `params.save()` to save these custom parameters to file. 
</div>

<div class="alert alert-block alert-success"><b>Note:</b> To see the contents of <code>params</code>, execute the cell below. Note, it can be large so caution is advised.</div>

In [6]:
print(params)

meerkat-src-100/config/params.config:
│   ├──  mpl-dir: ""
│   ├──  n-cpu: 8
│   ├──  paths: 
│   │   ├──  config: 
│   │   │   ├──  dir: meerkat-src-100/config
│   │   │   ├──  files: 
│   │   ├──  data: 
│   │   │   ├──  dir: meerkat-src-100/data
│   │   │   ├──  files: 
│   │   ├──  fits: 
│   │   │   ├──  dir: meerkat-src-100/data/fits
│   │   │   ├──  files: 
│   │   │   ├──  kalcal-diag: 
│   │   │   │   ├──  dir: meerkat-src-100/data/fits/kalcal-diag
│   │   │   │   ├──  files: 
│   │   │   │   ├──  filter: 
│   │   │   │   │   ├──  dir: meerkat-src-100/data/fits/kalcal-diag/filter
│   │   │   │   │   ├──  files: 
│   │   │   │   │   ├──  template: diag-filter-{itype}-{mp}mp-sigma_f-{sigma_f}.fits
│   │   │   │   ├──  smoother: 
│   │   │   │   │   └──  dir: meerkat-src-100/data/fits/kalcal-diag/smoother
│   │   │   │   │   └──  files: 
│   │   │   │   │   └──  template: diag-smoother-{itype}-{mp}mp-sigma_f-{sigma_f}.fits
│   │   │   ├──  kalcal-full: 
│   │   │   │   ├──  dir: 

In [ ]:
from source.script import all_settings

params, options = all_settings(params)

In [ ]:
from source.plotting import setup_plotting

# Setup plotting
setup_plotting(options["plots"], params)

In [ ]:
from source.data import create_empty_measurement_set

# Create measurement set
create_empty_measurement_set(options["ms"], params, verbose=True)

In [ ]:
from source.data import create_gains_signal

# Create true gains signal
create_gains_signal(options["gains"], params)

In [ ]:
from source.data import create_skymodels

# Create sky models
create_skymodels(options["vis"], params)

In [ ]:
from source.data import create_visibilities

# Create visibilities
create_visibilities(options["vis"], params)

In [ ]:
from source.plotting import create_amplitude_and_phase_signal_plot

# Plot amplitude and signal
create_amplitude_and_phase_signal_plot([0, 1, 2], params, show=True)

In [ ]:
from source.algorithms import run_kalcal_diag_calibration

run_kalcal_diag_calibration(options["kalcal-diag"], params, progress=True, check_mse=True)

In [ ]:
from source.algorithms import run_kalcal_full_calibration

run_kalcal_full_calibration(options["kalcal-full"], params, progress=True, check_mse=True)

In [ ]:
from source.algorithms import run_quartical_calibration

run_quartical_calibration(options["quartical"], params, progress=True, check_mse=True)

In [ ]:
from source.imaging import run_quartical_flux_extractor

run_quartical_flux_extractor(params, check_metric=True, progress=True)

In [ ]:
%autoreload now
from source.imaging import run_single_wsclean_imaging

paths = params["paths"]
gains_path = paths["gains"]["true"]
fits_name = "true"

run_single_wsclean_imaging(gains_path, fits_name, params)